In [450]:
import sys
sys.path.append('..')

import constants.file_handler_constants as fh
from constants.user_constants import *
from constants.attraction_constants import *

import os
import glob
import time
import pandas as pd
import numpy as np
import json
import requests
import ast
import random

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

from scipy.sparse import csr_matrix
import joblib

In [451]:
# Users with Similar Behavior to User 1: 2, 9, 11, 12, ...
# data = {
#     "attraction_id": ["A1", "A10", "A11", "A12", "A13", "A14", "A15", "A17", "A19", "A2", "A20", "A3", "A4", "A5", "A6", "A7", "A8", "A9"],
#     1:  [0.0, 1.0, 5.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0],
#     2:  [0.0, 1.0, 5.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0],
#     3:  [0.0, 4.0, 4.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#     4:  [5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 1.0],
#     5:  [0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
#     6:  [0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 3.0, 0.0],
#     7:  [0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 1.0, 0.0, 3.0, 0.0, 2.0, 0.0, 0.0, 2.0, 0.0],
#     8:  [0.0, 1.0, 0.0, 0.0, 0.0, 4.0, 0.0, 2.0, 0.0, 4.0, 2.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0],
#     9:  [0.0, 1.0, 5.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0],
#     10: [0.0, 0.0, 0.0, 5.0, 0.0, 4.0, 0.0, 0.0, 2.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0],
#     # 11: [0.0, 1.0, 5.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0],
#     # 12: [0.0, 1.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0],
#     # 13: [5.0, 4.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0],
#     # 14: [0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 3.0, 0.0, 1.0, 0.0],
#     # 15: [1.0, 2.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 2.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0],
#     # 16: [0.0, 1.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0],
#     # 17: [0.0, 1.0, 4.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 5.0, 0.0],
#     # 18: [4.0, 0.0, 0.0, 5.0, 0.0, 3.0, 2.0, 0.0, 0.0, 3.0, 5.0, 0.0, 1.0, 0.0, 0.0, 0.0, 3.0, 0.0],
#     # 19: [0.0, 0.0, 0.0, 5.0, 0.0, 5.0, 0.0, 0.0, 2.0, 0.0, 0.0, 5.0, 0.0, 4.0, 0.0, 0.0, 2.0, 0.0],
#     # 20: [0.0, 5.0, 5.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 1.0, 0.0, 2.0, 0.0, 4.0, 0.0],
# }



user_rating_data = [] # [[user_id_1, attraction_id_1, rating],...]
try:
    API_ENDPOINT = f"{os.environ['TRIPWEAVER_API']}/api/userrating/getAll"
    res_user_rating = requests.get(url=API_ENDPOINT).json()

    for cur_res_user_rating in res_user_rating:
        cur_user_id = cur_res_user_rating['userId']

        # user with no rating have no influence on this model
        if(not len(cur_res_user_rating['rating'])):
            continue
        
        for cur_user_rating in cur_res_user_rating['rating']:
            
            cur_attraction_id = cur_user_rating['attractionId']
            cur_rating_score = cur_user_rating['rating_score']

            user_rating_data.append([cur_user_id, cur_attraction_id, cur_rating_score])


except Exception as e:
    print("get user_rating failed !")
    print(e)





In [452]:
df = pd.DataFrame(user_rating_data, columns=['user_id', 'attraction_id', 'rating_score'])
df
# Rows = Users, Columns = Attractions
pivot_df = df.pivot_table(values = ['rating_score'], 
                       index = ['user_id'],
                       columns = ['attraction_id'],
                       aggfunc='last',
                       fill_value=0)

ratings_matrix = pivot_df
ratings_matrix

rating_score                           \
attraction_id            675c202f9f570448ebeb4829 675c202f9f570448ebeb482b   
user_id                                                                      
677e41b2519209b0aaff4d76                        5                        4   
677e41b6519209b0aaff4d7b                        5                        4   
677e41b9519209b0aaff4d80                        4                        4   
677e41bc519209b0aaff4d85                        4                        4   
677e41bf519209b0aaff4d8a                        4                        4   
677e41c2519209b0aaff4d8f                        4                        5   
677e41c5519209b0aaff4d94                        5                        5   
677e41c8519209b0aaff4d99                        4                        4   
677e41cb519209b0aaff4d9e                        5                        5   
677e41ce519209b0aaff4da3                        5                        5   
677e41d1519209b0aaff4da8                        5                        4   
677e41d4519209b0aaff4dad                        5                        4   
677e41d7519209b0aaff4db2                        4                        4   
677e41da519209b0aaff4db7                        5                        4   
677e41de519209b0aaff4dbc                        4                        5   
677e41df519209b0aaff4dc1                        5                        5   
677e41e1519209b0aaff4dc6                        5                        4   
677e41e2519209b0aaff4dcb                        5                        5   
677e41e3519209b0aaff4dd0                        4                        5   
677e41e5519209b0aaff4dd5                        5                        5   
677e4cd4519209b0aaff4f8c                        0                        0   
677e544b519209b0aaff4fd6                        0                        0   
677e544e519209b0aaff4fdb                        0                        0   
677e5451519209b0aaff4fe0                        0                        0   
677e5454519209b0aaff4fe5                        0                        0   
677e5457519209b0aaff4fea                        0                        0   
677e545a519209b0aaff4fef                        0                        0   
677e545d519209b0aaff4ff4                        0                        0   
677e5460519209b0aaff4ff9                        0                        0   
677e5464519209b0aaff4ffe                        0                        0   
677e5467519209b0aaff5003                        0                        0   
677e5469519209b0aaff5008                        0                        0   
677e546d519209b0aaff500d                        0                        0   
677e5470519209b0aaff5012                        0                        0   
677e5473519209b0aaff5017                        0                        0   
677e5476519209b0aaff501c                        0                        0   
677e547e519209b0aaff5027                        0                        0   
677e5484519209b0aaff5031                        0                        0   

                                                                            \
attraction_id            675c202f9f570448ebeb484b 675c202f9f570448ebeb484f   
user_id                                                                      
677e41b2519209b0aaff4d76                        4                        5   
677e41b6519209b0aaff4d7b                        4                        5   
677e41b9519209b0aaff4d80                        4                        4   
677e41bc519209b0aaff4d85                        5                        4   
677e41bf519209b0aaff4d8a                        4                        4   
677e41c2519209b0aaff4d8f                        4                        5   
677e41c5519209b0aaff4d94                        5                        5   
677e41c8519209b0aaff4d99                        4         

In [453]:
print(ratings_matrix.values)

[[5 4 4 ... 5 0 0]
 [5 4 4 ... 5 0 0]
 [4 4 4 ... 4 0 0]
 ...
 [0 0 0 ... 0 4 4]
 [0 0 0 ... 0 4 4]
 [0 0 0 ... 0 4 4]]


In [454]:
''' 
example user id of following persona (that have rating):
- user_persona_nature --> 677e41b2519209b0aaff4d76, 677e41b6519209b0aaff4d7b
- user_persona_art --> 677e544b519209b0aaff4fd6, 677e544e519209b0aaff4fdb

'''

# use '677e41b2519209b0aaff4d76' ('love_nature_0') for better example
# as this 'love_nature_0' have similar users rating art attractions that 'love_nature_0'  nerver been there and vote before

user_id = '677e41b6519209b0aaff4d7b'

knn = NearestNeighbors(metric='cosine')
knn.fit(ratings_matrix)

# Find the user's ratings
row_position = ratings_matrix.index.get_loc(user_id)
# Get 10 neighbors(10 similar user)
user_rating_vector = ratings_matrix.iloc[row_position, :].values.reshape(1,-1)
distances, indices = knn.kneighbors(user_rating_vector, n_neighbors=10+1)

print(indices)

# Get similar users(exclude the target user itself)
similar_users = indices.flatten()
similar_users = similar_users[similar_users != row_position]
print("check similar_users", similar_users)

# Aggregate attraction ratings from similar users
similar_users_ratings = ratings_matrix.iloc[similar_users].mean(axis=0)

# Exclude already rated attractions by the user
user_ratings = ratings_matrix.iloc[row_position]
recommendations = similar_users_ratings[user_ratings == 0]

# Sort recommendations by highest score
recommendations = recommendations.sort_values(ascending=False)
# save model
joblib.dump(knn, 'model_collaborative_filtering.joblib')


print(recommendations)
print("Idx of reccommed")
print(recommendations.reset_index()['attraction_id'].values)

[[ 1 16 19 18  2 17  0  9  3  6 13]]
check similar_users [16 19 18  2 17  0  9  3  6 13]
              attraction_id           
rating_score  675c20319f570448ebeb48d4    1.5
              675c20309f570448ebeb48be    1.4
              675c20349f570448ebeb49ac    1.4
              675c202f9f570448ebeb4867    1.3
              675c20319f570448ebeb48e4    1.3
              675c20309f570448ebeb48c6    1.3
              675c20379f570448ebeb4ac8    1.3
              675c20359f570448ebeb4a02    1.3
              675c20359f570448ebeb49d4    1.2
              675c20359f570448ebeb4a12    1.2
              675c20369f570448ebeb4a28    0.4
              675c20309f570448ebeb488d    0.0
              675c20309f570448ebeb4891    0.0
              675c20319f570448ebeb48cc    0.0
              675c20359f570448ebeb49c4    0.0
              675c20359f570448ebeb4a14    0.0
              675c20389f570448ebeb4af0    0.0
              675c20389f570448ebeb4afc    0.0
dtype: float64
Idx of reccommed
['675c20319f

In [455]:
recommendations.reset_index()['attraction_id'].values

array(['675c20319f570448ebeb48d4', '675c20309f570448ebeb48be',
       '675c20349f570448ebeb49ac', '675c202f9f570448ebeb4867',
       '675c20319f570448ebeb48e4', '675c20309f570448ebeb48c6',
       '675c20379f570448ebeb4ac8', '675c20359f570448ebeb4a02',
       '675c20359f570448ebeb49d4', '675c20359f570448ebeb4a12',
       '675c20369f570448ebeb4a28', '675c20309f570448ebeb488d',
       '675c20309f570448ebeb4891', '675c20319f570448ebeb48cc',
       '675c20359f570448ebeb49c4', '675c20359f570448ebeb4a14',
       '675c20389f570448ebeb4af0', '675c20389f570448ebeb4afc'],
      dtype=object)